# xdggs Healpix example

In [ ]:
import xarray as xr

import xdggs

Download the dataset here: https://zenodo.org/records/10075001

In [ ]:
ds = xr.open_dataset("data/healpix_nolotation.nc")

ds = (
    ds.load()
    .drop_vars(["latitude", "longitude"])
    .stack(cell=["x", "y"], create_index=False)
)

ds.cell_ids.attrs = {
    "grid_name": "healpix",
    "nside": 4096,
    "nest": True,
}

ds

In [ ]:
ds_idx = ds.set_xindex("cell_ids", xdggs.DGGSIndex)

ds_idx

## properties

cell boundaries

In [ ]:
cell_boundaries = ds_idx.dggs.cell_boundaries
cell_boundaries

In [ ]:
df = (
    cell_boundaries.to_dataset(name="geometry")
    .to_pandas()
    .set_geometry("geometry", crs=4326)
    .set_index("cell_ids")
)
df

In [ ]:
df.explore(tiles="OpenStreetMap")

## selection

by cell id

In [ ]:
ds_idx.sel(cell_ids=[11320973, 11320975])

by lat / lon coordinates

In [ ]:
ds_idx.dggs.sel_latlon([48.0, 48.1], -5.0)

## assignment

In [ ]:
ds2 = ds_idx.dggs.assign_latlon_coords()
ds2

In [ ]:
result = ds_idx.dggs.sel_latlon(ds2.latitude, ds2.longitude)
result

In [ ]:
xr.testing.assert_equal(result.drop_vars(["latitude", "longitude"]), ds)